In [1]:
#%load_ext tensorboard

In [2]:
import numpy as np
import pandas as pd
import importlib
import ipywidgets as widgets

import stable_baselines3 as sb
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [3]:
import cpagent
import cprender
import cpenvs

_ = importlib.reload(cpagent)
_ = importlib.reload(cprender)
_ = importlib.reload(cpenvs)

In [4]:
from stable_baselines3.common.env_checker import check_env
env = cpenvs.StandardCartPoleEnvCont(use_normalized_state=True)
check_env(env)

In [5]:
env = cpenvs.StandardCartPoleEnvCont(use_normalized_state=True)

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = sb.DDPG("MlpPolicy", env, action_noise=action_noise, verbose=0, tensorboard_log="./dqn_cartpole_tensorboard/")

In [6]:
model.learn(total_timesteps=5_000, log_interval=4, progress_bar=True)
model.save("ddpg_movingcont")

Output()

In [7]:
model = sb.DDPG.load("ddpg_movingcont")
agent = cpagent.ModelCartPoleAgent(model)

df = cpagent.execute_cartpole(agent, cpenvs.StandardCartPoleEnvCont())
df

,ep,t,cart_pos,cart_vel,pole_ang,pole_vel,pos_deviation,reward,cart_pos_setpoint
0,0,0,0.008980,-0.008670,0.038746,-0.044879,0.008980,NaN,0.0
1,0,1,0.008806,-0.185163,0.037849,0.231051,0.008806,1.0,0.0
2,0,2,0.005103,0.008583,0.042470,-0.048236,0.005103,1.0,0.0
3,0,3,0.005275,-0.169482,0.041505,0.231102,0.005275,1.0,0.0
4,0,4,0.001885,0.024105,0.046127,-0.046829,0.001885,1.0,0.0
...,...,...,...,...,...,...,...,...,...
2079,19,100,1.533979,1.552037,-0.168657,-0.541388,1.533979,1.0,0.0
2080,19,101,1.565019,1.510952,-0.179485,-0.529986,1.565019,1.0,0.0
2081,19,102,1.595238,1.467046,-0.190085,-0.517670,1.595238,1.0,0.0
2082,19,103,1.624579,1.420438,-0.200438,-0.504567,1.624579,1.0,0.0


In [8]:
_ = widgets.interact(
    lambda ep, t: cprender.render_cartpole_state_df(df, t, ep),
    ep=widgets.IntSlider(min=0, max=20, step=1, value=0),
    t=widgets.IntSlider(min=0, max=500, step=1, value=0),
)

c:\Source\studienarbeit-ml-controls\.env\lib\site-packages\ipywidgets\widgets\interaction.py:43: DeprecationWarning: `ipykernel.pylab.backend_inline` is deprecated, directly use `matplotlib_inline.backend_inline`
  from ipykernel.pylab.backend_inline import flush_figures


interactive(children=(IntSlider(value=0, description='ep', max=20), IntSlider(value=0, description='t', max=50…

In [9]:
EP = 0

fig = cprender.lineplot(df, ep=EP, incl_velo=True)
fig.show()